This function computes the Dynamic Polar Decomposition (DPD) of the $ 3\times 3$ deformation gradient $ \mathbf{\nabla F_{t_0}^t(\mathbf{x}_0)} $ for any times $ \tau, t \in [t_0, t_1] $:

\begin{equation}
\mathbf{\nabla F_{t_0}^t(\mathbf{x}_0)} = \mathbf{O}_{t_0}^t(\mathbf{x}_0)\mathbf{M}_{t_0}^t(\mathbf{x}_0) = \mathbf{N}_{t_0}^t(\mathbf{x}_0)\mathbf{O}_{t_0}^t(\mathbf{x}_0), 
\end{equation} where the proper orthogonal dynamic rotation tensor $ \mathbf{O}_{t_0}^t(\mathbf{x}_0) $ is the deformation gradient of a purely rotational flow; $ \mathbf{M}_{t_0}^t(\mathbf{x}_0) $ the right dynamic stretch tensor and the non-degenerate left dynamic stretch tensor $ \mathbf{N}_{t_0}^t(\mathbf{x}_0) $ are deformation gradients of purely straining flows.

The dynamic rotation tensor admits a further factorization:

\begin{equation}
\mathbf{O}_{t_0}^{t}(\mathbf{x}_0) = \mathbf{\Phi}_{t_0}^t(\mathbf{x}_0)\mathbf{\Theta}_{t_0}^t(\mathbf{x}_0)
\end{equation}

\begin{equation}
\mathbf{\dot{\Phi}}_{t_0}^t(\mathbf{x}_0) = [\mathbf{W}(\mathbf{F}_{t_0}^{t}(\mathbf{x}_0), t)-\mathbf{\overline{W}}(t)]\mathbf{\Phi}_{t_0}^t(\mathbf{x}_0), \quad \mathbf{\Phi}_{t_0}^{t_0}(\mathbf{x}_0) = \mathbf{I}
\label{eq: Phi_dot}
\end{equation}

\begin{equation}
\mathbf{\dot{\Theta}}_{t_0}^t(\mathbf{x}_0) = \mathbf{\Phi}_{t}^{t_0}(\mathbf{F}_{t_0}^{t}(\mathbf{x}_0))\mathbf{\overline{W}}(t)\mathbf{\Phi}_{t_0}^{t}(\mathbf{x}_0), \quad \mathbf{\Theta}_{t_0}^{t_0}(\mathbf{x}_0) = \mathbf{I}
\label{eq: Theta_dot}
\end{equation}

We start by computing $ \mathbf{\Phi}_{t_0}^t(\mathbf{x}_0) $ by solving the initial value problem (eq. \ref{eq: Phi_dot}) and then subsequently the initial value problem for $ \mathbf{\Theta}_{t_0}^{t} $ (eq. \ref{eq: Theta_dot}) over the interval $ t \in [t_0, t_N] $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| gradFmap | array (Nt, 3, 3) | $ \mathbf{\nabla F_{t_0}^{t}(\mathbf{x}_0)} $ |
| W | array (Nt, 3, 3) | $ \mathbf{W}(\mathbf{F}_{t_0}^{t}(\mathbf{x}_0), t) $ |
| W_avg | array (Nt, 3, 3) | $ \mathbf{\overline{W}}(t) $ |
| O | array (3, 3) | $ \mathbf{O}_{t_0}^t(\mathbf{x}_0) $|
| M | array (3, 3) | $ \mathbf{M}_{t_0}^t(\mathbf{x}_0) $|
| N | array (3, 3) | $ \mathbf{N}_{t_0}^t(\mathbf{x}_0) $|

In [3]:
# Import numpy
import numpy as np

# Import math tools
from math import sqrt, cos, sin, atan2

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [5]:
def DPD(gradFmap, W_avg, W, dt):
    
    THETA = np.eye(3,3) # Initial condition for Theta 
    
    Theta = []
    
    # solve IVP for THETA [t0, tN]
    for i in range(W_avg.shape[0]):
        Theta.append(THETA)
        THETA = THETA+(W[i,:,:]-W_avg[i,:,:])@THETA*dt
    
    Theta_memory = []
    
    # solve IVP for memory THETA
    for i in range(W_avg.shape[0]):
        
        THETA_memory = np.eye(3,3) # Initial condition for Theta

        for ii in range(i, 0, -1):
            THETA_memory = THETA_memory-(W[ii,:,:]-W_avg[ii,:,:])@THETA_memory*dt
        Theta_memory.append(THETA_memory)
        
    PHI = np.eye(3,3) # Initial condition for Phi
    
    Phi = []
    
    for i in range(W_avg.shape[0]):
        Phi.append(PHI)
        PHI = PHI+Theta_memory[i]@W_avg[i]@Theta[i]
        
    O, N, M = [], [], []
    
    for i in range(W_avg.shape[0]):
        Ot = Phi[i]@Theta[i]
        O.append(Ot)
        N.append(gradFmap[i,:,:]@np.linalg.inv(Ot))
        M.append(np.linalg.inv(Ot)@gradFmap[i,:,:])
    
    return np.array(Theta), np.array(Phi), np.array(O), np.array(M), np.array(N)